#Titanic - Machine Learning from a Disaster


In [1]:
import pandas
import numpy
import csv
import timeit
from sklearn import neighbors
from sklearn.neighbors import DistanceMetric 
from pprint import pprint
from math import sqrt
import operator

MY_TITANIC_TRAIN ='./sample_data/train.csv'
MY_TITANIC_TEST = './sample_data/test.csv'
titanic_dataframe = pandas.read_csv(MY_TITANIC_TRAIN)
titanic_test = pandas.read_csv(MY_TITANIC_TEST)
print('length: {0} '.format(len(titanic_dataframe)))
titanic_dataframe.head(4)
# titanic_dataframe = titanic_dataframe.head(50)

length: 891 


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [2]:
ave_age_train = titanic_dataframe['Age'].mean()
titanic_dataframe['Age'].fillna(ave_age_train, inplace=True)

ave_age_test = titanic_test['Age'].mean()
titanic_test['Age'].fillna(ave_age_test, inplace=True)

# forward filling 'Embarked' data under the assumption that consecutive passengers got on at the same port.
titanic_dataframe['Embarked'].fillna(method='ffill', inplace=True)
titanic_test['Embarked'].fillna(method='ffill', inplace=True)

We replaced the NA age values with the mean ages to complete data set.

In [3]:
titanic_dataframe['Gender'] = titanic_dataframe['Sex'].map({'female': 0, 'male': 1}).astype(int)
titanic_dataframe['Port'] = titanic_dataframe['Embarked'].map({'C':1, 'S':2, 'Q':3}).astype(int)
titanic_test['Gender'] = titanic_test['Sex'].map({'female': 0, 'male': 1}).astype(int)
titanic_test['Port'] = titanic_test['Embarked'].map({'C':1, 'S':2, 'Q':3}).astype(int)

titanic_dataframe.drop(['Sex', 'Embarked', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
titanic_test.drop(['Sex', 'Embarked', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [4]:
cols = titanic_dataframe.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]
titanic_dataframe = titanic_dataframe[cols]

train_data = titanic_dataframe[cols[2: ]]
train_target = titanic_dataframe[cols[1]]

pprint('column_list: {0}'.format(cols))
print(train_data.head(2))

("column_list: ['Survived', 'PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', "
 "'Fare', 'Gender', 'Port']")
   Pclass   Age  SibSp  Parch     Fare  Gender  Port
0       3  22.0      1      0   7.2500       1     2
1       1  38.0      1      0  71.2833       0     1


In [5]:
titanic_dataframe.head(5)

,Survived,PassengerId,Pclass,Age,SibSp,Parch,Fare,Gender,Port
0,0,1,3,22.0,1,0,7.2500,1,2
1,1,2,1,38.0,1,0,71.2833,0,1
2,1,3,3,26.0,0,0,7.9250,0,2
3,1,4,1,35.0,1,0,53.1000,0,2
4,0,5,3,35.0,0,0,8.0500,1,2


In [6]:
def hamming_distance(input_row, training_row):
    distance = list(
            1 if input_row[key] != training_row[key] else 0 for key in factors
    )
    return sum(distance)

factors = [ 'Pclass', 'Port', 'Gender']
hamming = DistanceMetric.get_metric('hamming')

In [7]:
def euclidian_distance(row, train_row):
    distance = list((((row[key] - train_row[key])**2)/titanic_dataframe[key].max()) for key in numericals)
#     print(sqrt(sum(distance)))
    return sqrt(sum(distance))
numericals = ['Age', 'SibSp', 'Parch', 'Fare']
euclidian = DistanceMetric.get_metric('euclidean')


In [ ]:
def knn(input_row, train, k):
    distance = []
    for row in train.iterrows():
        if row[0] != input_row[0]:
        # that means row = tuple and i can get what i want by saying row[1]['Column Name']            
            hamming = hamming_distance(input_row[1], row[1])
            euclidian = euclidian_distance(input_row[1], row[1])
            distance.append((hamming + euclidian, row[1]['Survived']))
    distance.sort(key=operator.itemgetter(0))
    out = distance[ :k]
    return (input_row[1]['PassengerId'], 1 if sum(out[1]) > k//2 else 0)

In [ ]:
predictions = open('predictions.csv', 'w')
file = csv.writer(predictions)
file.writerow(["PassengerId","Survived"])
for row in titanic_test.iterrows():
    output = knn(row, titanic_dataframe, 6)
    file.writerow(output)
predictions.close()